In [1]:
import seaborn as sns
import datetime 
import numpy as np
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import sklearn #(version == 1.2.2)
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import learning_curve
import os
%matplotlib inline

In [2]:
after_eda_dataset = pd.read_pickle(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\911Project\processed_datasets\after_eda_dataset.pkl")

In [3]:
ClusteredData = after_eda_dataset.drop(["title","timeStamp"],axis = 1)
ClusteredData

,lat,lng,desc,zip,twp,addr,e,EMT Reason,Hour,Month,Day of Week,Date
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,NEW HANOVER,REINDEER CT & DEAD END,1,EMS,17,12,Thu,2015-12-10
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS,17,12,Thu,2015-12-10
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,NORRISTOWN,HAWS AVE,1,Fire,17,12,Thu,2015-12-10
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,NORRISTOWN,AIRY ST & SWEDE ST,1,EMS,17,12,Thu,2015-12-10
5,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,LANSDALE,CANNON AVE & W 9TH ST,1,EMS,17,12,Thu,2015-12-10
...,...,...,...,...,...,...,...,...,...,...,...,...
99487,40.132869,-75.333515,MARKLEY ST & W LOGAN ST; NORRISTOWN; 2016-08-2...,19401.0,NORRISTOWN,MARKLEY ST & W LOGAN ST,1,Traffic,11,8,Wed,2016-08-24
99488,40.006974,-75.289080,LANCASTER AVE & RITTENHOUSE PL; LOWER MERION; ...,19003.0,LOWER MERION,LANCASTER AVE & RITTENHOUSE PL,1,Traffic,11,8,Wed,2016-08-24
99489,40.115429,-75.334679,CHESTNUT ST & WALNUT ST; NORRISTOWN; Station ...,19401.0,NORRISTOWN,CHESTNUT ST & WALNUT ST,1,EMS,11,8,Wed,2016-08-24
99490,40.186431,-75.192555,WELSH RD & WEBSTER LN; HORSHAM; Station 352; ...,19002.0,HORSHAM,WELSH RD & WEBSTER LN,1,EMS,11,8,Wed,2016-08-24


In [4]:
Y = ClusteredData["EMT Reason"]
ClusteredData = ClusteredData.drop(["EMT Reason","desc","addr",'Date'],axis = 1)
ClusteredData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86576 entries, 0 to 99491
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   lat          86576 non-null  float64
 1   lng          86576 non-null  float64
 2   zip          86576 non-null  float64
 3   twp          86576 non-null  object 
 4   e            86576 non-null  int64  
 5   Hour         86576 non-null  int64  
 6   Month        86576 non-null  int64  
 7   Day of Week  86576 non-null  object 
dtypes: float64(3), int64(3), object(2)
memory usage: 5.9+ MB


In [5]:
ClusteredData = ClusteredData.drop(['twp','Day of Week'],axis=1)

In [6]:
string_columns = ClusteredData.select_dtypes(include=['object', 'string']).columns
label_encoders = {}
for col in string_columns:
    le = LabelEncoder()
    ClusteredData[f'{col}_Encoded'] = le.fit_transform(ClusteredData[col])
    label_encoders[col] = le
    print(f"\nMapping for {col}:")
    for original, encoded in zip(le.classes_, range(len(le.classes_))):
        print(f"{original} -> {encoded}")
ClusteredData

,lat,lng,zip,e,Hour,Month
0,40.297876,-75.581294,19525.0,1,17,12
1,40.258061,-75.264680,19446.0,1,17,12
2,40.121182,-75.351975,19401.0,1,17,12
3,40.116153,-75.343513,19401.0,1,17,12
5,40.253473,-75.283245,19446.0,1,17,12
...,...,...,...,...,...,...
99487,40.132869,-75.333515,19401.0,1,11,8
99488,40.006974,-75.289080,19003.0,1,11,8
99489,40.115429,-75.334679,19401.0,1,11,8
99490,40.186431,-75.192555,19002.0,1,11,8


In [7]:
data, labels = ClusteredData, Y

X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size = 0.2, random_state = 42)

In [8]:
X_train.to_pickle(r'C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\911Project\processed_datasets\x_train.pkl')
X_test.to_pickle(r'C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\911Project\processed_datasets\x_test.pkl')
y_train.to_pickle(r'C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\911Project\processed_datasets\y_train.pkl')
y_test.to_pickle(r'C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\911Project\processed_datasets\y_test.pkl')